<a href="https://colab.research.google.com/github/hun9008/ML_TeamProject_24SS/blob/main/CNN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [31]:
import pickle

path = '/gdrive/My Drive/data/mature_train_labeled.pkl'

with open(path, 'rb') as f:
    mature = pickle.load(f)
    print('mature : ', len(mature))

mature :  2000


In [32]:
import pickle

path = '/gdrive/My Drive/data/incipient_train_labeled.pkl'

with open(path, 'rb') as f:
    incipient = pickle.load(f)
    print('incipient : ', len(incipient))

incipient :  2000


In [33]:
import numpy as np
from sklearn.model_selection import train_test_split

mature_train_labeled_list = []
for filename, data in mature.items():
    zero_centering_value = data.get('zero_centering')
    mature_train_labeled_list.append(np.array(zero_centering_value, dtype=np.float32))

# 리스트를 NumPy 배열로 변환
mature_train_labeled_value = np.array(mature_train_labeled_list)

print("mature_train_labeled_value shape:", mature_train_labeled_value.shape)

mature_train_labeled_value shape: (2000, 224, 224, 3)


In [34]:
import numpy as np
from sklearn.model_selection import train_test_split

incipient_train_labeled_list = []
for file, data in incipient.items():
    zero_centering_value = data.get('zero_centering')
    incipient_train_labeled_list.append(np.array(zero_centering_value, dtype=np.float32))


# 리스트를 NumPy 배열로 변환
incipient_train_labeled_value = np.array(incipient_train_labeled_list)

print("incipient_train_labeled_value shape:", incipient_train_labeled_value.shape)

incipient_train_labeled_value shape: (2000, 224, 224, 3)


In [35]:
X = np.concatenate((incipient_train_labeled_value, incipient_train_labeled_value), axis=0)
y = np.concatenate((np.zeros(incipient_train_labeled_value.shape[0]), np.ones(incipient_train_labeled_value.shape[0])))

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [36]:
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)

X_train shape: (2800, 224, 224, 3)
X_val shape: (600, 224, 224, 3)


In [43]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Reshape, Input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# 모델 설계
input_shape = (224, 224, 3)  # 이미지 크기와 채널 수 지정
input_layer = Input(shape=input_shape)
model = Sequential()
model.add(input_layer)

# CNN 부분
# 3개의 Conv2D 레이어와 MaxPooling2D, BatchNormalization 레이어를 차례로 쌓아 이미지의 특징을 추출
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

# 필터 수를 점점 늘리면서 더 복잡한 특징을 추출
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))

# Flatten 레이어 이후에 Reshape 레이어 추가
model.add(Reshape((1, 256)))

# LSTM 부분
# 첫 번째 LSTM은 시퀀스 데이터로 취급하고, 두 번째 LSTM은 시퀀스를 압축하여 최종 출력을 생성
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(256))

# 출력 층
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
# 시그모이드 활성화 함수를 사용하여 이진 분류(백내장 유무)를 수행
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))


Epoch 1/10
88/88 [==============================] - 10s 63ms/step - loss: 0.6996 - accuracy: 0.4807 - val_loss: 0.6940 - val_accuracy: 0.5000
Epoch 2/10
88/88 [==============================] - 4s 46ms/step - loss: 0.6993 - accuracy: 0.4911 - val_loss: 0.6975 - val_accuracy: 0.4983
Epoch 3/10
88/88 [==============================] - 4s 46ms/step - loss: 0.6962 - accuracy: 0.5061 - val_loss: 0.6936 - val_accuracy: 0.4950
Epoch 4/10
88/88 [==============================] - 4s 46ms/step - loss: 0.6950 - accuracy: 0.4911 - val_loss: 0.6950 - val_accuracy: 0.5017
Epoch 5/10
88/88 [==============================] - 4s 46ms/step - loss: 0.6945 - accuracy: 0.5068 - val_loss: 0.6932 - val_accuracy: 0.5183
Epoch 6/10
88/88 [==============================] - 4s 46ms/step - loss: 0.6945 - accuracy: 0.4807 - val_loss: 0.6945 - val_accuracy: 0.5017
Epoch 7/10
88/88 [==============================] - 4s 46ms/step - loss: 0.6933 - accuracy: 0.5079 - val_loss: 0.6946 - val_accuracy: 0.4583
Epoch 8/10
8

In [38]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_21 (Conv2D)          (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_21 (MaxPooli  (None, 111, 111, 32)      0         
 ng2D)                                                           
                                                                 
 batch_normalization_21 (Ba  (None, 111, 111, 32)      128       
 tchNormalization)                                               
                                                                 
 conv2d_22 (Conv2D)          (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_22 (MaxPooli  (None, 54, 54, 64)        0         
 ng2D)                                                           
                                                      



1.   softmax : 0.5 / 0.5
2.   sigmoid : 0.51 / 0.498



In [44]:
# 모델 평가
_, train_acc = model.evaluate(X_train, y_train, verbose=0)
_, val_acc = model.evaluate(X_val, y_val, verbose=0)

print('Train Accuracy: %.3f' % train_acc)
print('Validation Accuracy: %.3f' % val_acc)

Train Accuracy: 0.510
Validation Accuracy: 0.498
